# Section 1
Using ChatOpenAI for extraction.

In [13]:
!pip3 install openai langchain
!pip3 install -U langchain-openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_openai-0.0.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached tiktoken-0.5.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached regex-2023.12.25-cp39-cp39-macosx_11_0_arm64.whl.metadata (40 kB)
Using cached langchain_openai-0.0.5-py3-none-any.whl (29 kB)
Using cached tiktoken-0.5.2-cp39-cp39-macosx_11_0_arm64.whl (955 kB)
Using cached regex-2023.12.25-cp39-cp39-macosx_11_0_arm64.whl (291 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [29]:
import openai
import os
from langchain_openai import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


open_ai_api_key = open('../.open-ai-api-key').read().strip()
os.environ['OPENAI_API_KEY'] = open_ai_api_key

In [32]:
def text_to_cypher(text: str) -> str:
    chat = ChatOpenAI(temperature=0.7)

    example_text = """In this paper, we propose a novel method called reasoning on graphs (RoG) that synergizes LLMs
with KGs to conduct faithful and interpretable reasoning. To address the issues of hallucinations and
lack of knowledge, we present a planning-retrieval-reasoning framework, where RoG first generates
relation paths grounded by KGs as faithful plans via the planning module. These plans are then used
to retrieve valid reasoning paths from KGs to conduct faithful reasoning by the retrieval-reasoning
module. In this way, we not only retrieve the latest knowledge from KGs but also consider the
guidance of KG structure for reasoning and explanations. Moreover, the planning module of RoG
can be plug-and-play with different LLMs during inference to improve their performance. Based on
this framework, RoG is optimized by two tasks: 1) planning optimization, where we distill knowledge from KGs into LLMs to generate faithful relation paths as plans; and 2) retrieval-reasoning
optimization, where we enable LLMs to conduct faithful reasoning based on retrieved paths and generate interpretable results. We conduct extensive experiments on two benchmark KGQA datasets,
and the results demonstrate that RoG achieves state-of-the-art performance on KG reasoning tasks
and generates faithful and interpretable reasoning results.
"""

    system_msg = SystemMessage(content="You answer must include only code in Cypher without any comments or duplicate expressions.")
    first_shot_msg = HumanMessage(content=f'Your task is to extract objects and relationships' +
                     f'from the text in order to create a knowledge graph. Answer only with Cypher query that will put extracted' +
                     f'information into knowledge graph.\nThe example text:\n{example_text}')
    
    ai_example_msg = AIMessage(content="""CREATE (method:Method {name: "reasoning on graphs", description: "a novel method that synergizes LLMs with KGs to conduct faithful and interpretable reasoning"})
CREATE (framework:Framework {name: "planning-retrieval-reasoning", description: "a framework that generates relation paths grounded by KGs as faithful plans and retrieves valid reasoning paths from KGs to conduct faithful reasoning"})
CREATE (modulePlanning:Module {name: "planning", description: "a module that generates relation paths as plans"})
CREATE (modulePlanning)-[:PART_OF]->(framework)
CREATE (framework)-[:USES]->(method)
CREATE (moduleRetrievalReasoning:Module {name: "retrieval-reasoning", description: "a module that retrieves reasoning paths from KGs to conduct faithful reasoning"})
CREATE (moduleRetrievalReasoning)-[:PART_OF]->(framework)
CREATE (framework)-[:USES]->(method)
CREATE (moduleInference:Module {name: "inference", description: "a module that can be plug-and-play with different LLMs to improve their performance"})
CREATE (moduleInference)-[:PART_OF]->(method)
CREATE (benchmark:Dataset {name: "KGQA datasets", description: "benchmark datasets for KGQA"})
CREATE (taskPlanningOptimization:Task {name: "planning optimization", description: "optimization task where knowledge is distilled from KGs into LLMs to generate faithful relation paths as plans"})
CREATE (taskPlanningOptimization)-[:OPTIMIZES]->(method)
CREATE (taskRetrievalReasoningOptimization:Task {name: "retrieval-reasoning optimization", description: "optimization task where LLMs conduct faithful reasoning based on retrieved paths and generate interpretable results"})
CREATE (taskRetrievalReasoningOptimization)-[:OPTIMIZES]->(method)
CREATE (performanceSOTA:PerformanceMetric {name: "state-of-the-art performance", description: "achieving the best performance in KG reasoning tasks"})
CREATE (performanceSOTA)-[:ACHIEVED_BY]->(method)
CREATE (performanceFaithful:PerformanceMetric {name: "faithful and interpretable reasoning results", description: "generating reasoning results that are faithful and interpretable"})
CREATE (performanceFaithful)-[:ACHIEVED_BY]->(method)
""")
    main_msg = HumanMessage(content=f'Extract objects and relationships' +
                     f'from the text in order to create a knowledge graph. Answer only with Cypher query that will put extracted' +
                     f'information into knowledge graph.\nThe text to extract from:\n{text}')
    
    answer = chat.invoke([
        system_msg,
        first_shot_msg,
        ai_example_msg,
        main_msg
    ]).content

    return answer


In [33]:
text_for_extraction = """
Kelvin Kiptum Cheruiyot (2 December 1999 - 11 February 2024) was a Kenyan long-distance runner and the marathon world record holder at the time of his death. He is the only person in history to run the marathon in less than two hours and one minute in a record-eligible race. 
He ran three of the seven fastest marathons in history.
Kiptum won the three marathons he participated in, all renowned, including two top-tier World Marathon Majors (WMM), and held between December 2022 and October 2023. 
The times he achieved are three of the seven fastest times in history, a course record of less than 2:02 in each case, making him the only person ever to break this barrier thrice.
Kiptum ran the fastest-ever marathon debut at the 2022 Valencia Marathon, becoming only the third man in history to break two hours and two minutes and setting the then fourth-quickest time ever. 
He followed it up four months later with the second-fastest marathon in history at 2:01:25, 16 seconds outside the world record, at the 2023 London Marathon (WMM).
 In his next race, the 2023 Chicago Marathon (WMM) six months later in October 2023, he broke the world record by 34 seconds with a time of 2:00:35.
Kiptum died in a car crash in February 2024. At the time of his death, he was ranked first in the world in the men's marathon rankings.
"""
extracted_cypher = text_to_cypher(text_for_extraction)
print(extracted_cypher)

CREATE (runner:Runner {name: "Kelvin Kiptum Cheruiyot", nationality: "Kenyan", birth_date: date("1999-12-02"), death_date: date("2024-02-11")})
CREATE (record:Record {event: "marathon", time: "less than 2:02", record_type: "course record"})
CREATE (runner)-[:HOLDS_RECORD]->(record)
CREATE (achievement:Achievement {description: "the only person in history to run the marathon in less than two hours and one minute in a record-eligible race"})
CREATE (runner)-[:ACHIEVED]->(achievement)
CREATE (race1:Race {name: "Valencia Marathon 2022", date: date("2022-10-23"), time: "2:02", rank: "third"})
CREATE (runner)-[:PARTICIPATED_IN]->(race1)
CREATE (race2:Race {name: "London Marathon 2023", date: date("2023-04-23"), time: "2:01:25", rank: "second"})
CREATE (runner)-[:PARTICIPATED_IN]->(race2)
CREATE (race3:Race {name: "Chicago Marathon 2023", date: date("2023-10-23"), time: "2:00:35", rank: "first"})
CREATE (runner)-[:PARTICIPATED_IN]->(race3)
CREATE (ranking:Ranking {rank: 1, year: 2024, event: 

## Open questions
1. How to avoid creation of duplicate nodes (when we need to create new node and when we need to use existing)?
2. How to process big texts?

# section 2
Using pure llm endpoint for extraction. 

! to think about: create a tool that will help with conversion into cypher and create an agent for using it. 

In [34]:
from langchain_openai import OpenAI

def text_to_cypher_llm(text: str) -> str:
    prompt = """
    Given a detailed description of a research method, framework, and their components, generate Cypher queries to model these as nodes and relationships in a Neo4j graph database. For example:

    Description: In this paper, we propose a novel method called reasoning on graphs (RoG) that synergizes LLMs
with KGs to conduct faithful and interpretable reasoning. To address the issues of hallucinations and
lack of knowledge, we present a planning-retrieval-reasoning framework, where RoG first generates
relation paths grounded by KGs as faithful plans via the planning module. These plans are then used
to retrieve valid reasoning paths from KGs to conduct faithful reasoning by the retrieval-reasoning
module. In this way, we not only retrieve the latest knowledge from KGs but also consider the
guidance of KG structure for reasoning and explanations. Moreover, the planning module of RoG
can be plug-and-play with different LLMs during inference to improve their performance. Based on
this framework, RoG is optimized by two tasks: 1) planning optimization, where we distill knowledge from KGs into LLMs to generate faithful relation paths as plans; and 2) retrieval-reasoning
optimization, where we enable LLMs to conduct faithful reasoning based on retrieved paths and generate interpretable results. We conduct extensive experiments on two benchmark KGQA datasets,
and the results demonstrate that RoG achieves state-of-the-art performance on KG reasoning tasks
and generates faithful and interpretable reasoning results.
    Cypher Query: 
    CREATE (method:Method {name: "reasoning on graphs", description: "a novel method that synergizes LLMs with KGs to conduct faithful and interpretable reasoning"})
    CREATE (framework:Framework {name: "planning-retrieval-reasoning", description: "a framework that generates relation paths grounded by KGs as faithful plans and retrieves valid reasoning paths from KGs to conduct faithful reasoning"})
    [...]

    Now, based on the following description, generate the appropriate Cypher query:

    """ + f"{text}"
    
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        temperature=0.7,
        max_tokens=500,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response

In [35]:
extracted_cypher_pure_llm = text_to_cypher(text_for_extraction)
print(extracted_cypher_pure_llm)

CREATE (runner:Runner {name: "Kelvin Kiptum Cheruiyot", nationality: "Kenyan"})
CREATE (record:MarathonRecord {time: "less than two hours and one minute", date: "before February 2024"})
CREATE (runner)-[:HOLDS_RECORD]->(record)
CREATE (achievement:MarathonAchievement {description: "ran three of the seven fastest marathons in history"})
CREATE (runner)-[:ACHIEVES]->(achievement)
CREATE (marathon1:Marathon {name: "Valencia Marathon", year: 2022})
CREATE (marathon2:Marathon {name: "London Marathon", year: 2023})
CREATE (marathon3:Marathon {name: "Chicago Marathon", year: 2023})
CREATE (runner)-[:PARTICIPATED_IN]->(marathon1)
CREATE (runner)-[:PARTICIPATED_IN]->(marathon2)
CREATE (runner)-[:PARTICIPATED_IN]->(marathon3)
CREATE (marathon1)-[:FASTEST_TIME]->(time1:Time {time: "2:02"})
CREATE (marathon2)-[:FASTEST_TIME]->(time2:Time {time: "2:01:25"})
CREATE (marathon3)-[:FASTEST_TIME]->(time3:Time {time: "2:00:35"})
CREATE (achievement)-[:ACHIEVES_TIME]->(time1)
CREATE (achievement)-[:ACHIEV